# Oppimisprojekti 3: Mallien yhdistäminen

Tässä tehtävässä toteutetaan yksinkertainen "Puheesta generoitua tekstiä" - sovellus, joka ottaa syötteeksi äänitiedoston, transkriboi sen tekstiksi Whisper-mallilla ja käyttää tulosta tekstigeneraattorin syötteenä. Generaattori luo tämän pohjalta uutta tekstiä, joka muistuttaa koulutusdataa (tässä tapauksessa Tarzan-aiheista tekstiä).

#### 1. Transkriptio äänitiedostosta

Tässä vaiheessa käytetään OpenAI:n Whisper-mallia puheentunnistukseen. Malli ottaa syötteenä WAV-äänitiedoston (tarzan.wav) ja palauttaa tekstin.

In [ ]:
import whisper
from IPython.display import Audio, display

audio_file = "../week3_sounds/tarzan.wav"

model = whisper.load_model("large")

result = model.transcribe(audio_file)

with open("transkriptio.txt", "w") as f:
    f.write(result["text"])

display(Audio(audio_file, rate=16000))
print("Transkriptio:")
print(result["text"])

Transkriptio:
 Hän oli paljon nuorempi puolisoansa ja tietämättänsä hän oli kaivannut ystävyyden satamaan jonkun samanikäisemmän kanssa.


#### 2. Transkription lataus

Tiedostosta ladattu teksti tallennetaan muuttujaan seed, jota käytetään mallin lähtötekstinä.

In [12]:
with open("transkriptio.txt") as f:
    seed = f.read().strip()

#### 3. Mallin lataus

Ladataan aiemmin koulutettu kielimalli sekä siihen liittyvä SentencePiece-tokenisaattori.

In [ ]:
from keras.models import load_model

model = load_model("tarzan_best_model.keras", compile=False)

In [14]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.load("tarzan_sp.model")

True

#### 4. Tekstin siistiminen ennen tulostusta

Generoitu teksti voi sisältää ylimääräisiä välimerkkejä tai puuttuvia isoja alkukirjaimia yms. Tämä funktio huolehtii tekstin ulkonäöstä ja parantaa luettavuutta.

In [ ]:
import re

def clean_generated_text(text):
    # Ylimääräiset välilyönnit pois
    text = re.sub(r"\s+", " ", text)

    # Poista lainausmerkkejä ja ylimääräisiä välimerkkejä
    text = re.sub(r'["“”]', "", text)
    text = re.sub(r'[^\w\s\.\,\-\?!"\'äöÄÖ]', "", text)

    # Erotetaan lauseet pisteistä yms
    sentences = re.split(r"(?<=[\.\?\!])\s+", text)

    # Lauseet isolla alkukirjaimella ja lyhyet lauseet pois
    sentences = [s.strip().capitalize() for s in sentences if len(s.strip()) > 10]

    # Yhdistetään kokonaiseksi tekstiksi
    cleaned = " ".join(sentences)
    return cleaned

#### 5. Tekstigeneraattorin toteutus

Toteutetaan tekstin generointi. Malli saa lähtötekstin (seed) ja tuottaa siihen jatkoa token kerrallaan. Tulokseen vaikuttavat kaksi parametria: tekstin pituus ja lämpötila.

Temperature-parametri säätelee, kuinka satunnaista generoitu teksti on:

- Matala arvo (esim. 0.2–0.5):
    - Malli valitsee todennäköisimmät sanat, jolloin teksti on ennustettavaa.
- Keskitaso (esim. 0.7–1.0):
    - Lopputulos on tasapaino ennustettavuuden ja luovuuden välillä.
- Korkea arvo (1.2+):
    - Teksti on luovempaa, mutta saattaa muuttua epäloogiseksi ja kaoottiseksi.

Myös tekstin pituus on säädettävissä. Tässä oletuksena käytettiin 100 tokenia, mutta tätä voi kasvattaa tai pienentää, kuten lämpötilaakin.

In [ ]:
import numpy as np

def generate_text(model, sp, seed_text, length=100, temperature=1.0):
    model_input = list(map(int, sp.encode_as_ids(seed_text)))

    for _ in range(length):
        input_ids = model_input[-64:]
        if len(input_ids) < 64:
            input_ids = [0] * (64 - len(input_ids)) + input_ids
        input_array = np.array([input_ids], dtype=np.int32)
        prediction = model.predict(input_array, verbose=0)
        logits = prediction[0, -1]
        logits = logits / temperature
        exp_preds = np.exp(logits - np.max(logits))
        probs = exp_preds / np.sum(exp_preds)
        next_token_id = np.random.choice(len(probs), p=probs)
        model_input.append(int(next_token_id))

    return sp.decode_ids(model_input)

print("Seed:")
print(seed)

generated = generate_text(model, sp, seed, length=150, temperature=0.9)
generated = clean_generated_text(generated)
print("\nGeneroitu teksti:")
print(generated)

Seed:
Hän oli paljon nuorempi puolisoansa ja tietämättänsä hän oli kaivannut ystävyyden satamaan jonkun samanikäisemmän kanssa.

Generoitu teksti:
Hän oli paljon nuorempi puolisoansa ja tietämättänsä hän oli kaivannut ystävyyden satamaan jonkun samanikäisemmän kanssa. Nainen katsahti häneen odottaen näkevänsä murhanhimon vimmatun välkkeen silmissä yläpuolellaan. Sensijaan hän näkikin surua ja katumusta. Huudahti hän. Katsokaa, mitä olette tehnyt. Hän oli puolisoni. Minä rakastin häntä ja te olette hänet tappanut. Hyvin hellävaroen nosti tarzan kreivi de couden hervottoman ruumiin ja kantoi sen vuoteelle. Sitten hän painoi korvansa miehen rintaa vasten. Hiukan konjakkia, olga! Kreivitär toi sitä, ja yhdessä he kaatoivat sitä hänen huuliensa välitse. Samalla nämä valkoiset huulet heikosti hengähtivät. Pää kääntyi, ja de coude voihkaisi. Hän ei kuole, sanoi tarzan. Jumalan kiitos! Miksi niin teitte, jean? Kysyi kreivitär. Hän löi minua ja minä vimmastuin. Olen nähnyt heimoni apinain tekev

#### 6. Päätelmä

Tässä tehtävässä yhdistettiin kaksi eri mallia: generatiivinen tekstimalli ja puheentunnistus. Äänitiedostona käytetty puhe valittiin niin, että sen tyyli olisi lähellä tekstigeneraattorin koulutusaineistoa, jotta mallin olisi helpompi tuottaa yhteensopivaa jatkotekstiä.

Vaikka generoitu teksti ei ollut niinkään korkeatasoista, sen tyyli vastasi yllättävän hyvin koulutusaineistoa. Tämä osoittaa, että malli on oppinut koulutusaineiston kielen rakenteen ja tyylin melko hyvin. Puheesta saatiin tuotettua tyylillisesti uskottavaa jatkotekstiä.